In [97]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline
import seaborn as sns
import pandas_profiling
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score, log_loss
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.feature_selection import SelectKBest 
from sklearn.feature_selection import chi2
import collections
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from collections import Counter


In [98]:
x1=pd.ExcelFile(r'Data_Train.xlsx')
x2=pd.ExcelFile(r'Data_Test.xlsx')

In [99]:
columns_read=['Name','Location','Year','Kilometers_Driven','Fuel_Type','Transmission','Owner_Type','Mileage','Engine','Power'
             ,'Seats','New_Price','Price']
columns_readt=['Name','Location','Year','Kilometers_Driven','Fuel_Type','Transmission','Owner_Type','Mileage','Engine','Power'
             ,'Seats','New_Price']

In [100]:
ds_input = x1.parse(u'Sheet1',names=columns_read)
dst_input = x2.parse(u'Sheet1',names=columns_readt)

In [101]:
ds_input =ds_input.drop_duplicates()
ds_input.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6019 entries, 0 to 6018
Data columns (total 13 columns):
Name                 6019 non-null object
Location             6019 non-null object
Year                 6019 non-null int64
Kilometers_Driven    6019 non-null int64
Fuel_Type            6019 non-null object
Transmission         6019 non-null object
Owner_Type           6019 non-null object
Mileage              6017 non-null object
Engine               5983 non-null object
Power                5983 non-null object
Seats                5977 non-null float64
New_Price            824 non-null object
Price                6019 non-null float64
dtypes: float64(2), int64(2), object(9)
memory usage: 658.3+ KB


In [102]:
y_train = ds_input['Price']
train_nomissing = ds_input.drop('Price',axis=1)
test_nomissing = dst_input
combined_set = pd.concat([train_nomissing, test_nomissing], axis = 0) # Stacks them vertically

In [103]:
 combined_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7253 entries, 0 to 1233
Data columns (total 12 columns):
Name                 7253 non-null object
Location             7253 non-null object
Year                 7253 non-null int64
Kilometers_Driven    7253 non-null int64
Fuel_Type            7253 non-null object
Transmission         7253 non-null object
Owner_Type           7253 non-null object
Mileage              7251 non-null object
Engine               7207 non-null object
Power                7207 non-null object
Seats                7200 non-null float64
New_Price            1006 non-null object
dtypes: float64(1), int64(2), object(9)
memory usage: 736.6+ KB


In [104]:
combined_set["Power"] = combined_set["Power"].str.replace('null bhp','0 bhp')
combined_set["Power"].fillna('0 bhp',inplace=True)
combined_set["Mileage"].fillna('0.0 kmpl',inplace=True)
combined_set["Engine"].fillna('0 CC',inplace=True)
combined_set["Seats"].fillna(0,inplace=True)

In [105]:
#spliting mileage
ds_mil = combined_set["Mileage"].str.split(" ",n=1,expand=True)
ds_mil1 = ds_mil[0].str.split(".",n=1,expand=True)
combined_set["Mileage"]=ds_mil1[0].astype(str).astype(int)

In [106]:
ds_eng = combined_set["Engine"].str.split(" ",n=1,expand=True)
combined_set["Engine"]=ds_eng[0].astype(str).astype(int)

In [107]:
ds_pwr = combined_set["Power"].str.split(" ",n=1,expand=True)
ds_pwr1 = ds_pwr[0].str.split(".",n=1,expand=True)

combined_set["Power"]=ds_pwr1[0].astype(str).astype(int)

In [108]:
combined_set["New_Price"]= combined_set["New_Price"].replace(np.nan,'0 Lakh')

ds_prc = combined_set["New_Price"].str.split(" ",n=1,expand=True)
combined_set["cur"]=ds_prc[1].astype(str)

In [109]:
ds_prc1 = ds_prc[0].str.split(".",n=1,expand=True)
combined_set["New_Price"]=ds_prc1[0].astype(str).astype(int)

In [110]:
def cal_cur(val_cur):
    val,cur = val_cur
    if cur=='Cr':
        val = val*100
    return val

combined_set["New_Price"] = combined_set[["New_Price",'cur']].apply(cal_cur, axis=1)

In [111]:
combined_set["Name2"]=combined_set["Name"].apply(lambda x:' '.join(x.split()[:2]))
combined_set["Name1"]=combined_set["Name"].apply(lambda x:' '.join(x.split()[:1]))

In [112]:
combined_set['Name2'] = combined_set['Name2'].apply(lambda x: " ".join(x.lower() for x in x.split()))
combined_set['Name'] = combined_set['Name'].apply(lambda x: " ".join(x.lower() for x in x.split()))
combined_set['Name1'] = combined_set['Name1'].apply(lambda x: " ".join(x.lower() for x in x.split()))


In [113]:
combined_set.head(3)

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,cur,Name2,Name1
0,maruti wagon r lxi cng,Mumbai,2010,72000,CNG,Manual,First,26,998,58,5.0,0,Lakh,maruti wagon,maruti
1,hyundai creta 1.6 crdi sx option,Pune,2015,41000,Diesel,Manual,First,19,1582,126,5.0,0,Lakh,hyundai creta,hyundai
2,honda jazz v,Chennai,2011,46000,Petrol,Manual,First,18,1199,88,5.0,8,Lakh,honda jazz,honda


In [114]:
combined_set["Power"].replace(0,np.nan,inplace=True)
combined_set["Mileage"].replace(0,np.nan,inplace=True)
combined_set["Engine"].replace(0,np.nan,inplace=True)
combined_set["Seats"].replace(0,np.nan,inplace=True)
combined_set["New_Price"].replace(0,np.nan,inplace=True)

In [115]:
by_Name=combined_set.groupby(['Name'])
by_Name2=combined_set.groupby(['Name2'])
by_Name1=combined_set.groupby(['Name1'])

In [116]:
def impute_median(series):
    return series.fillna(series.median())

In [117]:
combined_set["Power"]=by_Name['Power'].transform(impute_median)
combined_set["Mileage"]=by_Name['Mileage'].transform(impute_median)
combined_set["Engine"]=by_Name['Engine'].transform(impute_median)
combined_set["Seats"]=by_Name['Seats'].transform(impute_median)
combined_set["New_Price"]=by_Name['New_Price'].transform(impute_median)

In [118]:
combined_set["Power"]=by_Name2['Power'].transform(impute_median)
combined_set["Mileage"]=by_Name2['Mileage'].transform(impute_median)
combined_set["Engine"]=by_Name2['Engine'].transform(impute_median)
combined_set["Seats"]=by_Name2['Seats'].transform(impute_median)
combined_set["New_Price"]=by_Name2['New_Price'].transform(impute_median)

In [119]:
combined_set["Power"]=by_Name1['Power'].transform(impute_median)
combined_set["Mileage"]=by_Name1['Mileage'].transform(impute_median)
combined_set["Engine"]=by_Name1['Engine'].transform(impute_median)
combined_set["Seats"]=by_Name1['Seats'].transform(impute_median)

In [120]:
combined_set['Mileage'].fillna(0,inplace= True)
combined_set['Power'].fillna(0,inplace= True)
combined_set['New_Price'].fillna(0,inplace= True)

In [121]:
dummy1_combined_set=combined_set

### Data exploration

In [122]:
df_train =  pd.concat([combined_set[:train_nomissing.shape[0]],y_train], axis=1)
dst_input = combined_set[train_nomissing.shape[0]:]

In [123]:
df_train['Kilometers_Driven'] =df_train['Kilometers_Driven'].apply(lambda x: 650000 if x>6000000 else x)
df_train =df_train.drop(df_train.loc[(df_train['Fuel_Type']=='Electric') ].index.tolist()).reset_index(drop=True) 
df_train =df_train.drop(df_train.loc[(df_train['Name1']=='porsche') & (df_train['Price']<3)].index.tolist()).reset_index(drop=True)

In [124]:
df_train['namelen'] = [min(70, len(n)) for n in df_train['Name']]
dst_input['namelen'] = [min(70, len(n)) for n in dst_input['Name']]

In [125]:
price_df_train = pd.DataFrame(df_train)
price_df_train.to_excel('price_df_train.xlsx')

In [126]:
#combining the train and test 
y_train = df_train['Price']
train_nomissing = df_train.drop('Price',axis=1)
test_nomissing = dst_input
combined_set = pd.concat([train_nomissing, test_nomissing], axis = 0) # Stacks them vertically

In [127]:
dummy2_combined_set=combined_set
combined_set = dummy2_combined_set

In [128]:
# agegroup below 30 and above 59 as a categorical feature
def premium(pro):
    if(pro in ["maruti","hyundai",'honda','ford','mahindra','tata','skoda','renault'
              ,'chevrolet','nissan','fiat','datsun','ambassador','hindustan'
              ,'Smart']):
        pro='I'
    else:
        pro='P'
    return pro
      
combined_set['is_premium'] = combined_set['Name1'].apply(premium)

In [129]:
# agegroup below 30 and above 59 as a categorical feature
def Exp_output(Ex):
    if Ex<=2006:
        Ex=2006
    return Ex
      
combined_set['Year'] = combined_set['Year'].apply(Exp_output)

In [130]:
# agegroup below 30 and above 59 as a categorical feature
def Exp_output(Ex):
    if (Ex in ['CNG','LPG']):
        Ex='LPG_CNG'
    return Ex
      
combined_set['Fuel_Type'] = combined_set['Fuel_Type'].apply(Exp_output)

In [131]:
# agegroup below 30 and above 59 as a categorical feature
def Exp_owner(Ex_Tr):
    Ex,Tr=Ex_Tr
    if ((Tr =='Manual' and Ex in ['Second','Third']) or (Ex=='Fourth & Above')):
        Ex='com'
    return Ex
      
combined_set['Owner_Type'] = combined_set[['Owner_Type','Transmission']].apply(Exp_owner,axis=1)

In [132]:
# agegroup below 30 and above 59 as a categorical feature
def Exp_price(Ex_pr):
    Ex,pr=Ex_pr
    if Ex in ['mercedes-benz s','jaguar f'] :
        pr=200
    elif Ex in ['audi a8','mercedes-benz sl-class']:
        pr=100
    elif Ex in ['mercedes-benz gl-class','bmw x6','jaguar xj','mercedes-benz slk-class','volvo xc90','audi a7',
               'porsche cayman','toyota land'] :
        pr=90
    elif Ex in ['bmw x5','bmw 6','mitsubishi montero']:
        pr=70
    elif Ex in ['mercedes-benz m-class','audi tt','bmw z4'] :
        pr=66
    elif Ex in ['audi q5','volvo xc60','mercedes-benz r-class','volvo s80','nissan 370z']:
        pr=55
    elif Ex=='honda accord':
        pr=44
    elif Ex in ['hyundai santa','volkswagen passat','renault koleos','nissan x-trail','chevrolet captiva','bmw 1',
               'mitsubishi outlander','fiat abarth']:
        pr=30
    elif Ex in[ 'mahindra ssangyong','skoda yeti','nissan teana','maruti grand','isuzu mu']:
        pr= 24
    elif Ex=='hyundai sonata':
        pr=18
    elif Ex in ['volkswagen jetta','isuzu d-max']:
        pr=17
    elif Ex in['skoda laura','chevrolet cruze','renault fluence']:
        pr=16.5
    elif Ex=='tata new':
        pr=14
    elif Ex in ['mahindra scorpio','renault scala','tata xenon','force one','chevrolet tavera','renault captur']:
        pr=12
    elif Ex in ['ford fiesta','maruti sx4','honda civic','mahindra tuv','chevrolet optra','maruti s',
               'tata sumo','mahindra quanto','toyota qualis','volkswagen crosspolo','renault lodgy','mitsubishi lancer'
               ,'mitsubishi cedia','mahindra jeep']:
        pr=9
    elif Ex in ['hyundai accent','honda mobilio','chevrolet enjoy','renault pulse','fiat grande','mahindra renault',
               'ford classic']:
        pr=7
    elif Ex in ['maruti ritz','tata manza','chevrolet aveo','chevrolet sail','maruti esteem','mahindra logan',
               'ford fusion','opelcorsa 1.4gsi']:
        pr=6
    elif Ex in ['tata indigo','hyundai i10','ford ikon','fiat petra','hindustan motors']:
        pr= 5
    elif Ex in ['hyundai eon','hyundai getz','chevrolet beat','tata venture']:
        pr=4.5
    elif Ex in['tata indica','maruti a-star','maruti estilo','maruti versa','maruti 1000','datsun redi']:
        pr=4
    elif Ex in ['chevrolet spark','maruti zen']:
        pr=3.5
    elif Ex in ['maruti omni','maruti 800']:
        pr=3
    elif Ex =='tata nano':
        pr=2.5     
    return pr
      
combined_set['New_Price'] = combined_set[['Name2','New_Price']].apply(Exp_price,axis=1)

In [133]:
combined_set=combined_set.drop(['Name','Name1','cur'],axis=1)
combined_set=pd.get_dummies(data=combined_set)

In [134]:
st_dummies = pd.get_dummies(combined_set.Seats, prefix='Seats')
combined_set = pd.concat([combined_set,st_dummies], axis=1)

combined_set=combined_set.drop(['Seats'],axis=1)

In [135]:
df_train = combined_set[:train_nomissing.shape[0]]
df_test = combined_set[train_nomissing.shape[0]:]

In [136]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
df_train_arr_scld = sc.fit_transform(df_train)
df_train_scld=pd.DataFrame(df_train_arr_scld,columns=df_train.columns)
df_test_arr_scld = sc.transform(df_test)
df_test_scld=pd.DataFrame(df_test_arr_scld,columns=df_test.columns)

In [137]:
train = df_train_scld 
test = df_test_scld 

labels = y_train

In [138]:
train.head(2)

,Year,Kilometers_Driven,Mileage,Engine,Power,New_Price,namelen,Location_Ahmedabad,Location_Bangalore,Location_Chennai,...,is_premium_I,is_premium_P,Seats_2.0,Seats_4.0,Seats_5.0,Seats_6.0,Seats_7.0,Seats_8.0,Seats_9.0,Seats_10.0
0,-1.099422,0.368090,1.937179,-1.036946,-1.009167,-0.652672,-0.523170,-0.196657,-0.251542,-0.298769,...,0.643324,-0.643324,-0.05164,-0.12935,0.436555,-0.07197,-0.355501,-0.150935,-0.022336,-0.028841
1,0.510872,-0.434202,0.266755,-0.063648,0.254823,-0.211941,0.733173,-0.196657,-0.251542,-0.298769,...,0.643324,-0.643324,-0.05164,-0.12935,0.436555,-0.07197,-0.355501,-0.150935,-0.022336,-0.028841


In [139]:
print(train.shape)
print(test.shape)

(6016, 257)
(1234, 257)


In [140]:
train1=train.copy()
test1=test.copy()
y_train1=y_train.copy()

In [141]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR

In [142]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [143]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

### Model Building

In [144]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,max_depth=4,
                                   min_samples_leaf=15, min_samples_split=15, loss='huber', random_state =5)

In [145]:
GBoost.fit(train, y_train)
gb_train_pred = GBoost.predict(train)

In [146]:
gb_pred = (GBoost.predict(test.values))

In [147]:
print(rmsle(y_train, gb_train_pred))

1.7306934682038493


In [148]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.52, gamma=0.03, 
                             learning_rate=0.072, max_depth=6, 
                             min_child_weight=2, n_estimators=2200,
                             reg_alpha=0, reg_lambda=1,
                             subsample=0.615, silent=1,
                             random_state =7, nthread = -1)

In [149]:
model_xgb.fit(train, y_train)
xgb_train_pred = model_xgb.predict(train)
xgb_pred = model_xgb.predict(test)
print(rmsle(y_train, xgb_train_pred))

0.31700792810615586


In [150]:
price_XG_pred = pd.DataFrame(xgb_pred)
price_XG_pred.to_excel('price_test_reslt_xgb1.xlsx')

In [151]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=31,
                              learning_rate=0.05, n_estimators=2800,feature_fraction = 0.75,
                              feature_fraction_seed=9,min_data_in_leaf =15
                                                         
                              )

In [152]:
model_lgb.fit(train, y_train)
lgb_train_pred = model_lgb.predict(train)

In [153]:
print(rmsle(y_train, lgb_train_pred))

0.33650575458555665


In [154]:
lgb_pred = model_lgb.predict(test.values)
price_pred_lgb = pd.DataFrame(lgb_pred)
price_pred_lgb.to_excel('price_test_reslt_lgbm.xlsx')

In [155]:
ens_train =  xgb_train_pred*0.4 + lgb_train_pred*0.3 + gb_train_pred*0.3
ens_train_pd = pd.DataFrame(ens_train)
ens_train_pd.to_excel('ens_train_pd.xlsx')

In [156]:
'''RMSE on the entire Train data when averaging'''

print('RMSLE score on train data:')
print(rmsle(y_train,xgb_train_pred*0.4 + lgb_train_pred*0.3 + gb_train_pred*0.3 ))

RMSLE score on train data:
0.645398613241735


In [157]:
ensemble =  xgb_pred*0.4 + lgb_pred*0.3 + gb_pred*0.3
price_pred_ensb = pd.DataFrame(ensemble)
price_pred_ensb.to_excel('price_test_reslt_ensb1.xlsx')